In [1]:
import time
import copy
import itertools

In [2]:
M = 9
grid = [[0, 7, 0, 0, 0, 4, 5, 0, 0],
        [0, 8, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 7, 8, 0],
        [0, 0, 0, 0, 0, 9, 0, 2, 0],
        [0, 0, 0, 5, 0, 0, 0, 0, 4],
        [9, 0, 6, 0, 0, 0, 0, 0, 1],
        [0, 0, 3, 0, 2, 0, 0, 0, 0],
        [0, 1, 2, 0, 4, 0, 0, 0, 0],
        [0, 0, 0, 8, 7, 5, 0, 0, 0]]

In [3]:
def puzzle(a):
    for i in range(M):
        for j in range(M):
            print(a[i][j],end = " ")
        print()

In [4]:
# Check whether a specific number can be used for specific dimensions
def isValid(board, num, pos):

    row, col = pos
    # Check if all row elements include this number
    for j in range(9):
        if board[row][j] == num:
            return False

    # Check if all column elements include this number
    for i in range(9):
        if board[i][col] == num:
            return False

    # Check if the number is already included in the block
    rowBlockStart = 3* (row // 3)
    colBlockStart = 3* (col // 3)

    rowBlockEnd = rowBlockStart + 3
    colBlockEnd = colBlockStart + 3
    for i in range(rowBlockStart, rowBlockEnd):
        for j in range(colBlockStart, colBlockEnd):
            if board[i][j] == num:
                return False

    return True

In [5]:
def Suduko(grid, row, col):
 
    if (row == M - 1 and col == M):
        return True
    if col == M:
        row += 1
        col = 0
    if grid[row][col] > 0:
        return Suduko(grid, row, col + 1)
    for num in range(1, M + 1, 1): 
     
        if isValid(grid, num, (row, col)):
         
            grid[row][col] = num
            if Suduko(grid, row, col + 1):
                return True
        grid[row][col] = 0
    return False

In [6]:
start_time = time.time()
'''0 means the cells where no value is assigned'''
grid_solve = copy.deepcopy(grid)
if (Suduko(grid_solve, 0, 0)):
    puzzle(grid_solve)
else:
    print("Solution does not exist:(")

print("--- %s seconds ---" % (time.time() - start_time))

6 7 1 2 8 4 5 3 9 
3 8 5 6 9 7 1 4 2 
2 9 4 1 5 3 7 8 6 
5 3 8 4 1 9 6 2 7 
1 2 7 5 6 8 3 9 4 
9 4 6 7 3 2 8 5 1 
7 5 3 9 2 1 4 6 8 
8 1 2 3 4 6 9 7 5 
4 6 9 8 7 5 2 1 3 
--- 2.900547504425049 seconds ---


In [7]:
# Better solution

In [8]:
# Store in a dictionary the legitimate values for each individual cell
def cacheValidValues(board):
    cache = dict()
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                cache[(i,j)] = allowedValues(board,i,j)
    return cache

In [9]:
def allowedValues_old(board,row,col):

    numbersList = list()

    for number in range(1,10):

        found = False
        # Check if all row elements include this number
        for j in range(9):
            if board[row][j] == number:
                found = True
                break
        # Check if all column elements include this number
        if found == True:
            continue
        else:
            for i in range(9):
                if board[i][col] == number:
                    found = True
                    break

        # Check if the number is already included in the block
        if found == True:
            continue
        else:
            rowBlockStart = 3* (row // 3)
            colBlockStart = 3* (col // 3)

            rowBlockEnd = rowBlockStart + 3
            colBlockEnd = colBlockStart + 3
            for i in range(rowBlockStart, rowBlockEnd):
                for j in range(colBlockStart, colBlockEnd):
                    if board[i][j] == number:
                        found = True
                        break
        if found == False:
            numbersList.append(number)
    return numbersList

In [10]:
def allowedValues(board,row,col):

    numbersList = list([i for i in range(1,10)])
    
    SquareRow = (row // 3)*3
    squareColumn = (col // 3)*3
    numbers = board[int(row)] + [board[r][col] for r in range(9)] + [board[y][z] for y in range(SquareRow, SquareRow+3) for z in range(squareColumn, squareColumn+3)]
    numbers = list(set(numbers))
    if 0 in numbers:
        numbers.remove(0)
    for number in numbers:
        numbersList.remove(number)
    
    return numbersList

In [11]:
# Find next empty space in Sudoku board and return dimensions
def findEmpty(board):
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0 :
                return row,col
    return None

In [12]:
def solveWithCache(board, cache):
    blank = findEmpty(board)
    if not blank:
        return True
    else:
        row, col = blank

    for value in cache[(row,col)]:
        if isValid(board, value, blank):
            board[row][col] = value

            if solveWithCache(board, cache):
                return True

            board[row][col] = 0
    return False

In [13]:
# Find the case with the least possibilities ro reduce time
def solveWithCache_v2(board):
    cache = cacheValidValues(board)
    if not cache:
        return True
    else:
        ((row,col),liste) = min(cache.items(), key=lambda x: len(x[1]))

    for value in liste:
        board[row][col] = value

        if solveWithCache_v2(board):
            return True

        board[row][col] = 0
    return False

In [14]:
# Making use of the available information on each row, change the order that
# each value will be 'guessed' by our algorithm
def orderedValidValues_v2(board, cache):
    cachePriority = dict()
    countAppearanceRow = [dict() for i in range(9)]
    countAppearanceCol = [dict() for i in range(9)]
    countAppearanceBlock = [dict() for i in range(9)]
    for row in range(9):
        tempList = list()

        # Iterate through the columns of a row and count appearance of numbers
        # within the cache
        for col in range(9):
            if (row,col) in cache.keys():
                for value in cache[(row,col)]:
                    tempList.append(value)
        tempSet = set(tempList)
        for number in tempSet:
            countAppearanceRow[row][number] = tempList.count(number)


    for col in range(9):
        tempList = list()

        # Iterate through the rows of a column and count appearance of numbers
        # within the cache
        for row in range(9):
            if (row,col) in cache.keys():
                for value in cache[(row,col)]:
                    tempList.append(value)
        tempSet = set(tempList)
        for number in tempSet:
            countAppearanceCol[col][number] = tempList.count(number)

    # Iterate through the 9 different blocks of the board and count
    # appearance of numbers within the cache
    rowBlockStart = 0
    colBlockStart = 0
    blockNumber = 0
    while True:

        rowBlockEnd = rowBlockStart + 3
        colBlockEnd = colBlockStart + 3
        tempList = list()
        for row in range(rowBlockStart,rowBlockEnd):
            for col in range(colBlockStart,colBlockEnd):
                if (row,col) in cache.keys():
                    for value in cache[(row,col)]:
                        tempList.append(value)
        tempSet = set(tempList)
        for number in tempSet:
            countAppearanceBlock[blockNumber][number] = tempList.count(number)
        if rowBlockStart == 6 and colBlockStart == 6:
            break
        elif colBlockStart == 6:
            rowBlockStart += 3
            colBlockStart = 0
        else:
            colBlockStart += 3
        blockNumber += 1

    for row in range(9):
        for col in range(9):
            tempList = list()
            blockNumber = (row // 3) * 3 + col // 3
            if (row,col) in cache.keys():
                for value in cache[(row,col)]:
                    freq = countAppearanceRow[row][value] + countAppearanceCol[col][value] + countAppearanceBlock[blockNumber][value]
                    tempList.append(freq)
                cachePriority[(row,col)] = tempList


    for row in range(9):
        for col in range(9):
            if (row,col) in cache.keys():
                cache[(row,col)] = [i for _,i in sorted(zip(cachePriority[(row,col)], cache[(row,col)]))]
    

    return cache

In [15]:
start_time = time.time()
'''0 means the cells where no value is assigned'''


grid_solve = copy.deepcopy(grid)
cache = cacheValidValues(grid_solve)
if (solveWithCache(grid_solve,cache)):
    puzzle(grid_solve)
else:
    print("Solution does not exist:(")
print("--- %s seconds ---" % (time.time() - start_time))

6 7 1 2 8 4 5 3 9 
3 8 5 6 9 7 1 4 2 
2 9 4 1 5 3 7 8 6 
5 3 8 4 1 9 6 2 7 
1 2 7 5 6 8 3 9 4 
9 4 6 7 3 2 8 5 1 
7 5 3 9 2 1 4 6 8 
8 1 2 3 4 6 9 7 5 
4 6 9 8 7 5 2 1 3 
--- 2.436713933944702 seconds ---


In [16]:
start_time = time.time()
'''0 means the cells where no value is assigned'''


grid_solve = copy.deepcopy(grid)
if (solveWithCache_v2(grid_solve)):
    puzzle(grid_solve)
else:
    print("Solution does not exist:(")
print("--- %s seconds ---" % (time.time() - start_time))

6 7 1 2 8 4 5 3 9 
3 8 5 6 9 7 1 4 2 
2 9 4 1 5 3 7 8 6 
5 3 8 4 1 9 6 2 7 
1 2 7 5 6 8 3 9 4 
9 4 6 7 3 2 8 5 1 
7 5 3 9 2 1 4 6 8 
8 1 2 3 4 6 9 7 5 
4 6 9 8 7 5 2 1 3 
--- 0.024935483932495117 seconds ---


In [153]:
# Recursive + iterative

In [247]:
# Making use of the available information on each row, change the order that
# each value will be 'guessed' by our algorithm
def orderedValidValues_v3(board, cache):
    cachePriority = dict()
    countAppearanceRow = [dict.fromkeys([j for j in range(1,10)], 0) for i in range(9)]
    countAppearanceCol = [dict.fromkeys([j for j in range(1,10)], 0) for i in range(9)]
    countAppearanceBlock = [dict.fromkeys([j for j in range(1,10)], 0) for i in range(9)]
    
    
    for (row,col) in cache.keys():
        values = cache[(row,col)]
        blockNumber = (row // 3) * 3 + col // 3
        for value in values:
            countAppearanceRow[row][value] += 1
            countAppearanceCol[col][value] += 1
            countAppearanceBlock[blockNumber][value] += 1
            

    boolean = False
    for (row,col) in cache.keys():
        values = cache[(row,col)]
        blockNumber = (row // 3) * 3 + col // 3
        for value in values:
            if countAppearanceRow[row][value] == 1 or countAppearanceCol[col][value] == 1 or countAppearanceBlock[blockNumber][value] == 1:
                board[row][col] = value
                boolean = True
    return boolean

In [248]:
start_time = time.time()
'''0 means the cells where no value is assigned'''


grid_solve = copy.deepcopy(grid)
valuesFound = True
while valuesFound:
    cacheValid = cacheValidValues(grid_solve)
    valuesFound = orderedValidValues_v3(grid_solve, cacheValid)
#     puzzle(grid_solve)
#     print("\n\n")

if (solveWithCache_v2(grid_solve)):
    puzzle(grid_solve)
else:
    print("Solution does not exist:(")
print("--- %s seconds ---" % (time.time() - start_time))

6 7 1 2 8 4 5 3 9 
3 8 5 6 9 7 1 4 2 
2 9 4 1 5 3 7 8 6 
5 3 8 4 1 9 6 2 7 
1 2 7 5 6 8 3 9 4 
9 4 6 7 3 2 8 5 1 
7 5 3 9 2 1 4 6 8 
8 1 2 3 4 6 9 7 5 
4 6 9 8 7 5 2 1 3 
--- 0.028921842575073242 seconds ---
